# Python packages
 
Creating a python environment:
```bash
conda create -y -n dda_validate -c conda-forge python \
ipython \
jupyter \
numpy \
numba \
pyparsing \
pillow \
shapely \
scipy \
matplotlib
```
Activate our environment:

```bash
conda activate dda_validate
```

Installing python packages:
```bash
pip install miepython
```
```bash
pip install PyMieScatt
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import miepython
import bhmie
import PyMieScatt as ps

In [ ]:
def plot_efficiency_factros(m,x):
    # NOTE: np.conj(m)
    qext, qsca, qback, g = miepython.mie(np.conj(m), x)
    qabs = qext - qsca

    plt.plot(x, qsca, color='red', label='Qsca')
    # plt.plot(x, qext, color='green', label='Qext')
    plt.plot(x, qabs, color='blue', label='Qabs')

    plt.ylabel("efficiency factor")
    plt.yscale("log")
    plt.ylim([1e-3, max([max(qsca), max(qabs)])])
    plt.xlabel("x=ka")
    plt.legend()

    plt.show()

    print('max:' ,np.max(qsca), 'ind: ', np.argmax(qsca))
    print('min:' ,np.min(qsca), 'ind: ', np.argmin(qsca))



def plot_efficiency_factros_ps(m,x):

    xValues, qext, qsca, qabs, g, qpr, qback, qratio = ps.MieQ_withSizeParameterRange(m, nMedium=1.0, xRange=(min(x), max(x)), nx=len(x), logX=False)

    plt.plot(xValues, qsca, color='red', label='Qsca')
    # plt.plot(xValues, qext, color='green', label='Qext')
    plt.plot(xValues, qabs, color='blue', label='Qabs')

    plt.ylabel("efficiency factor")
    plt.yscale("log")
    plt.ylim([1e-3, max([max(qsca), max(qabs)])])
    plt.xlabel("x=ka")
    plt.legend()

    plt.show()

    print('max:' ,np.max(qsca), 'ind: ', np.argmax(qsca))
    print('min:' ,np.min(qsca), 'ind: ', np.argmin(qsca))

def plot_efficiency_factros_bhmie(m,x):

    # s1,s2,qext,qsca,qback,gsca = bhmie(x,m, n)

    qext, qsca = np.zeros_like(x), np.zeros_like(x)
    for (i, xi) in enumerate(x):
        _, _, qext[i], qsca[i], _, _ = bhmie.bhmie(xi, m, 2)
        
    qabs = qext - qsca

    plt.plot(x, qsca, color='red', label='Qsca')
    # plt.plot(x, qext, color='green', label='Qext')
    plt.plot(x, qabs, color='blue', label='Qabs')

    plt.ylabel("efficiency factor")
    plt.yscale("log")
    plt.ylim([1e-3, max([max(qsca), max(qabs)])])
    plt.xlabel("x=ka")
    plt.legend()

    plt.show()

    print('max:' ,np.max(qsca), 'ind: ', np.argmax(qsca))
    print('min:' ,np.min(qsca), 'ind: ', np.argmin(qsca))




$$ \frac{\mathrm{d}\, Q}{\mathrm{d}\, \Omega} = \frac{S_{11}}{\pi(ka)^2} $$

In [ ]:
def plot_differential_scattering_efficiencies(m, x):
    # NOTE: np.conj(m)
    # NOTE: normalised s1, s2 

    theta = np.linspace(0, np.pi, 2*180+1)
    mu = np.cos(theta)

    scat_unp = miepython.i_unpolarized(np.conj(m), x, mu)
    scat_par = miepython.i_par(m, x, mu)
    scat_per = miepython.i_per(m, x, mu)

    qext, qsca, qback, g = miepython.mie(np.conj(m), x)

    # print(np.pi * qext * x**2 / (np.pi * x**2))
    scat_unp *=  qext 

    angles = theta/np.pi*180
    plt.plot(angles, scat_unp, label='unpolarised')
    # plt.plot(angles, scat_par, label='paralel')
    # plt.plot(angles, scat_per, label='perpendicular')

    plt.plot(angles, qsca/(4*np.pi)*np.ones_like(angles))

    plt.ylabel('differential scattering efficiencies')
    plt.yscale("log")
    # plt.ylim([5e-3, max(scat)])
    plt.xlabel("theta")
    plt.legend()

    plt.show()

    print('max:' ,np.max(scat_unp), 'ind: ', angles[np.argmax(scat_unp)])
    print('min:' ,np.min(scat_unp), 'ind: ', angles[np.argmin(scat_unp)])


def plot_differential_scattering_efficiencies_ps(m, x):

    wavelength = 2 * np.pi
    diameter = 2 * x
    measure, SL, SR, SU = ps.ScatteringFunction(m, wavelength, diameter, nMedium=1.0, minAngle=0, maxAngle=180, angularResolution=.5, space='theta', angleMeasure='radians', normalization=None)

    SU /= np.pi * x**2

    angles = measure/np.pi*180
    plt.plot(angles, SU, label='unpolarised')
    # plt.plot(angles, SR, label='paralel')
    # plt.plot(angles, SL, label='perpendicular')

    qext, qsca, qabs, g, qpr, qback, qratio = ps.MieQ(m, wavelength, diameter, nMedium=1.0, asDict=False, asCrossSection=False)
    plt.plot(angles, qsca/(4*np.pi)*np.ones_like(angles))

    plt.ylabel('differential scattering efficiencies')
    plt.yscale("log")
    # plt.ylim([5e-3, max(scat)])
    plt.xlabel("theta")
    plt.legend()

    plt.show()
    print('max:' ,np.max(SU), 'ind: ', angles[np.argmax(SU)])
    print('min:' ,np.min(SU), 'ind: ', angles[np.argmin(SU)])

def plot_differential_scattering_efficiencies_bhmie(m, x):

    N = 181
    theta = np.linspace(0, np.pi, 2*N-1)

    s1, s2, qext, qsca, qback, gsca = bhmie.bhmie(x, m, N)

    scat_par = np.abs(s2)**2
    scat_per = np.abs(s1)**2
    scat_unp = (scat_par+scat_per)/2

    scat_unp /= np.pi * x**2


    angles = theta/np.pi*180
    plt.plot(angles, scat_unp, label='unpolarised')
    # plt.plot(angles, scat_par, label='paralel')
    # plt.plot(angles, scat_per, label='perpendicular')

    plt.plot(angles, qsca/(4*np.pi)*np.ones_like(angles))


    plt.ylabel('differential scattering efficiencies')
    plt.yscale("log")
    # plt.ylim([5e-3, max(scat)])
    plt.xlabel("theta")
    plt.legend()

    plt.show()
    print('max:' ,np.max(scat_unp), 'ind: ', angles[np.argmax(scat_unp)])
    print('min:' ,np.min(scat_unp), 'ind: ', angles[np.argmin(scat_unp)])

In [ ]:
m = 1.33+.01j
x = np.linspace(0.0001, 13, 1001)

plot_efficiency_factros(m,x)
plot_efficiency_factros_ps(m,x)
plot_efficiency_factros_bhmie(m,x)


In [ ]:
m = 2+1j
x = np.linspace(0.0001, 13, 1001)

plot_efficiency_factros(m,x)
plot_efficiency_factros_ps(m,x)
plot_efficiency_factros_bhmie(m,x)


In [ ]:
m = 1.33+0.01j
x = 3

plot_differential_scattering_efficiencies(m, x)
plot_differential_scattering_efficiencies_ps(m, x)
plot_differential_scattering_efficiencies_bhmie(m, x)

In [ ]:
m = 1.33+0.01j
x = 5

plot_differential_scattering_efficiencies(m, x)
plot_differential_scattering_efficiencies_ps(m, x)
plot_differential_scattering_efficiencies_bhmie(m, x)

In [ ]:
m = 1.33+0.01j
x = 7

plot_differential_scattering_efficiencies(m, x)
plot_differential_scattering_efficiencies_ps(m, x)
plot_differential_scattering_efficiencies_bhmie(m, x)

In [ ]:
m = 2+1j
x = 3

plot_differential_scattering_efficiencies(m, x)
plot_differential_scattering_efficiencies_ps(m, x)
plot_differential_scattering_efficiencies_bhmie(m, x)

In [ ]:
m = 2+1j
x = 5

plot_differential_scattering_efficiencies(m, x)
plot_differential_scattering_efficiencies_ps(m, x)
plot_differential_scattering_efficiencies_bhmie(m, x)

In [ ]:
m = 2+1j
x = 7

plot_differential_scattering_efficiencies(m, x)
plot_differential_scattering_efficiencies_ps(m, x)
plot_differential_scattering_efficiencies_bhmie(m, x)